In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from datetime import datetime as dt
import random
from functools import reduce

In [2]:
pd.set_option('display.max_colwidth', None, "display.max_rows", None, "display.max_columns", None)
pd.options.mode.chained_assignment = None

In [3]:
config = "1tls_3x3"

In [4]:
dfs = [
    pd.read_csv(f'./{config}/con/DuelingDoubleDQNAgent_lr0.0001.csv', delimiter=',').assign(id_con=lambda df: df['id']+'_con', con=True),
    pd.read_csv(f'./{config}/con/MaxPressureBaseline.csv', delimiter=',').assign(id_con=lambda df: df['id']+'_con', con=True),
    # pd.read_csv(f'./{config}/con/SotlBaseline.csv', delimiter=',').assign(id_con=lambda df: df['id']+'_con', con=True),
    # pd.read_csv(f'./{config}/con/UniformBaseline.csv', delimiter=',').assign(id_con=lambda df: df['id']+'_con', con=True),
    pd.read_csv(f'./{config}/def/DuelingDoubleDQNAgent_lr0.0001.csv', delimiter=',').assign(id_con=lambda df: df['id']+'_def', con=False),
    pd.read_csv(f'./{config}/def/MaxPressureBaseline.csv', delimiter=',').assign(id_con=lambda df: df['id']+'_def', con=False),
    pd.read_csv(f'./{config}/def/SotlBaseline.csv', delimiter=',').assign(id_con=lambda df: df['id']+'_def', con=False),
    pd.read_csv(f'./{config}/def/UniformBaseline.csv', delimiter=',').assign(id_con=lambda df: df['id']+'_def', con=False),   
]
None

In [5]:
for df in dfs:
    display(df.loc[df["done"] == True].head(2))

,l,r,id,ep,con_p_rate,ctrl_con_p_rate,veh_n_p_hour,veh_n,sum_delay,sum_waiting_time,sum_acc_waiting_time,sum_queue_length,avg_delay,avg_waiting_time,avg_acc_waiting_time,avg_queue_length,TimeLimit.truncated,done,id_con,con
301,302,91.929191,rlcontroller,1,0.237,0.237,"[1000.0, 200.0, 500.0, 200.0]",52,44.746220,5515.0,5676.0,38,0.860504,106.057692,109.153846,3.166667,False,True,rlcontroller_con,True
613,312,193.264708,rlcontroller,2,0.983,0.983,"[100.0, 200.0, 200.0, 600.0]",29,23.767757,3074.0,3195.0,20,0.819578,106.000000,110.172414,1.666667,False,True,rlcontroller_con,True


,l,r,id,ep,con_p_rate,ctrl_con_p_rate,veh_n_p_hour,veh_n,sum_delay,sum_waiting_time,sum_acc_waiting_time,sum_queue_length,avg_delay,avg_waiting_time,avg_acc_waiting_time,avg_queue_length,TimeLimit.truncated,done,id_con,con
252,253,0.0,maxpressurebaseline,1,0.237,0.237,"[1000.0, 200.0, 500.0, 200.0]",22,14.815938,206.0,264.0,6,0.673452,9.363636,12.000000,0.500000,False,True,maxpressurebaseline_con,True
498,246,0.0,maxpressurebaseline,2,0.983,0.983,"[100.0, 200.0, 200.0, 600.0]",13,7.746416,54.0,54.0,4,0.595878,4.153846,4.153846,0.333333,False,True,maxpressurebaseline_con,True


,l,r,id,ep,con_p_rate,ctrl_con_p_rate,veh_n_p_hour,veh_n,sum_delay,sum_waiting_time,sum_acc_waiting_time,sum_queue_length,avg_delay,avg_waiting_time,avg_acc_waiting_time,avg_queue_length,TimeLimit.truncated,done,id_con,con
287,288,90.115140,rlcontroller,1,1.0,0.237,"[1000.0, 200.0, 500.0, 200.0]",25,17.236500,3400.0,3428.0,10,0.689460,136.000000,137.120000,0.833333,False,True,rlcontroller_def,False
600,313,138.392722,rlcontroller,2,1.0,0.983,"[100.0, 200.0, 200.0, 600.0]",31,25.713759,3557.0,3728.0,22,0.829476,114.741935,120.258065,1.833333,False,True,rlcontroller_def,False


,l,r,id,ep,con_p_rate,ctrl_con_p_rate,veh_n_p_hour,veh_n,sum_delay,sum_waiting_time,sum_acc_waiting_time,sum_queue_length,avg_delay,avg_waiting_time,avg_acc_waiting_time,avg_queue_length,TimeLimit.truncated,done,id_con,con
244,245,0.0,maxpressurebaseline,1,1.0,0.237,"[1000.0, 200.0, 500.0, 200.0]",20,12.137099,200.0,200.0,4,0.606855,10.00,10.00,0.333333,False,True,maxpressurebaseline_def,False
489,245,0.0,maxpressurebaseline,2,1.0,0.983,"[100.0, 200.0, 200.0, 600.0]",12,7.051218,33.0,33.0,2,0.587602,2.75,2.75,0.166667,False,True,maxpressurebaseline_def,False


,l,r,id,ep,con_p_rate,ctrl_con_p_rate,veh_n_p_hour,veh_n,sum_delay,sum_waiting_time,sum_acc_waiting_time,sum_queue_length,avg_delay,avg_waiting_time,avg_acc_waiting_time,avg_queue_length,TimeLimit.truncated,done,id_con,con
265,266,0.0,sotlbaseline,1,1.0,0.237,"[1000.0, 200.0, 500.0, 200.0]",139,128.723890,1388.0,19946.0,76,0.926071,9.985612,143.496403,6.333333,False,True,sotlbaseline_def,False
533,268,0.0,sotlbaseline,2,1.0,0.983,"[100.0, 200.0, 200.0, 600.0]",74,69.428477,644.0,14430.0,46,0.938223,8.702703,195.000000,3.833333,False,True,sotlbaseline_def,False


,l,r,id,ep,con_p_rate,ctrl_con_p_rate,veh_n_p_hour,veh_n,sum_delay,sum_waiting_time,sum_acc_waiting_time,sum_queue_length,avg_delay,avg_waiting_time,avg_acc_waiting_time,avg_queue_length,TimeLimit.truncated,done,id_con,con
239,240,0.0,uniformbaseline,1,1.0,0.237,"[1000.0, 200.0, 500.0, 200.0]",110,102.707175,2078.0,17894.0,87,0.933702,18.890909,162.672727,7.250000,False,True,uniformbaseline_def,False
479,240,0.0,uniformbaseline,2,1.0,0.983,"[100.0, 200.0, 200.0, 600.0]",76,71.851957,789.0,18240.0,46,0.945420,10.381579,240.000000,3.833333,False,True,uniformbaseline_def,False


In [6]:
def sum_ep(df):
    return pd.merge(
        df[['id', 'ep', 'veh_n', 'sum_delay', 'sum_waiting_time', 'sum_acc_waiting_time', 'sum_queue_length']].groupby(['ep']).sum().rename(columns={"veh_n": "sum_veh_n"}),
        df[['id', 'ep', 'con_p_rate', 'ctrl_con_p_rate', 'veh_n_p_hour', 'id_con', 'con']].groupby(['ep']).apply(lambda df: df.head(1)),
        how='left', left_index=True, right_index=True
    ).reset_index(drop=True)

def sum_df_ep(dfs):
    return pd.concat([sum_ep(df) for df in dfs]).sort_values(['ep', 'id']).reset_index(drop=True)

def avg_ep(df):
    return pd.merge(
        df[['id', 'ep', 'veh_n', 'avg_delay', 'avg_waiting_time', 'avg_acc_waiting_time', 'avg_queue_length']].groupby(['ep']).mean().rename(columns={"veh_n": "avg_veh_n"}),
        df[['id', 'ep', 'con_p_rate', 'ctrl_con_p_rate', 'veh_n_p_hour', 'id_con', 'con']].groupby(['ep']).apply(lambda df: df.head(1)),
        how='left', left_index=True, right_index=True
    ).reset_index(drop=True)

def avg_df_ep(dfs):
    return pd.concat([avg_ep(df) for df in dfs]).sort_values(['ep', 'id']).reset_index(drop=True)

def make_df_ep(dfs):
    return pd.merge(
        sum_df_ep(dfs),
        avg_df_ep(dfs)[['avg_veh_n', 'avg_delay', 'avg_waiting_time', 'avg_acc_waiting_time', 'avg_queue_length']],
        how='left', left_index=True, right_index=True
    ).assign(ctrl_con_p_rate_round=lambda df: (df['ctrl_con_p_rate'] * 10).astype(int) / 10).sort_index(axis=1)

In [7]:
ep_df = make_df_ep(dfs)

In [8]:
display(ep_df.head(12))

,avg_acc_waiting_time,avg_delay,avg_queue_length,avg_veh_n,avg_waiting_time,con,con_p_rate,ctrl_con_p_rate,ctrl_con_p_rate_round,ep,id,id_con,sum_acc_waiting_time,sum_delay,sum_queue_length,sum_veh_n,sum_waiting_time,veh_n_p_hour
0,22.043245,0.718157,0.920290,27.154150,20.712891,True,0.237,0.237,0.2,1,maxpressurebaseline,maxpressurebaseline_con,160153.0,4976.620533,2794,6870,150257.0,"[1000.0, 200.0, 500.0, 200.0]"
1,9.055964,0.687747,0.702381,24.269388,8.749424,False,1.000,0.237,0.2,1,maxpressurebaseline,maxpressurebaseline_def,54718.0,4117.327792,2065,5946,52766.0,"[1000.0, 200.0, 500.0, 200.0]"
2,56.913785,0.809477,2.006623,41.807947,51.523649,True,0.237,0.237,0.2,1,rlcontroller,rlcontroller_con,744542.0,10357.084276,7272,12626,670686.0,"[1000.0, 200.0, 500.0, 200.0]"
3,57.294971,0.771721,1.430556,34.000000,49.240486,False,1.000,0.237,0.2,1,rlcontroller,rlcontroller_def,563542.0,7619.617137,4944,9792,484093.0,"[1000.0, 200.0, 500.0, 200.0]"
4,123.209393,0.907000,5.472118,97.755639,12.063332,False,1.000,0.237,0.2,1,sotlbaseline,sotlbaseline_def,3818007.0,24214.236350,17467,26003,339909.0,"[1000.0, 200.0, 500.0, 200.0]"
5,140.100095,0.909160,4.882639,88.179167,12.994183,False,1.000,0.237,0.2,1,uniformbaseline,uniformbaseline_def,3356150.0,19630.189134,14062,21163,287377.0,"[1000.0, 200.0, 500.0, 200.0]"
6,6.551360,0.671868,0.426152,16.077236,6.401381,True,0.983,0.983,0.9,2,maxpressurebaseline,maxpressurebaseline_con,26513.0,2680.369170,1258,3955,25885.0,"[100.0, 200.0, 200.0, 600.0]"
7,6.699139,0.674553,0.427891,16.183673,6.545513,False,1.000,0.983,0.9,2,maxpressurebaseline,maxpressurebaseline_def,27059.0,2698.722914,1258,3965,26410.0,"[100.0, 200.0, 200.0, 600.0]"
8,79.389989,0.813574,1.442842,28.423077,66.359207,True,0.983,0.983,0.9,2,rlcontroller,rlcontroller_con,734947.0,7264.769800,5402,8868,613352.0,"[100.0, 200.0, 200.0, 600.0]"
9,72.990590,0.809140,1.396699,27.859425,65.062860,False,1.000,0.983,0.9,2,rlcontroller,rlcontroller_def,663901.0,7103.427341,5246,8720,592638.0,"[100.0, 200.0, 200.0, 600.0]"


In [9]:
###################################################################################################################

In [10]:
ep_df.to_csv(f'./data/{config}.csv', sep = ',')